# Building impacted by war fires
# ==============================

This notebook focuses on analyzing the impact of war fires on buildings in Ukraine. It begins by importing necessary libraries and setting a development flag. The script then retrieves war fire data from a GitHub repository and reads both war fire and building datasets. Utilizing reverse geocoding, it determines the state of each building based on its geographical coordinates. The processed war fire data is saved, and subsequent steps involve filtering the data for specific Ukrainian states, creating a geometry column, and converting coordinates to the UTM coordinate system. Additionally, the code calculates the centroid of buildings and performs a spatial analysis to count the number of buildings within a 50-meter buffer of each war fire. The final results, detailing the impact of war fires on buildings, are saved to a CSV file. The script concludes by exporting the code as a Python script for future use.

In [1]:
# Auto update notebook imports
#%load_ext autoreload
#%autoreload 2

# Backtrack to folder source directory if it doesn't already exist in path
import os
import sys

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

sys.path.append(os.getcwd())
print(os.getcwd())

import geopandas as gpd
import pandas as pd
from shapely.wkt import loads
import numpy as np
from shapely import wkt
from shapely.geometry import Point
from tqdm.notebook import tqdm
from urllib.request import urlretrieve

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="myexer")

from src.data import make_dataset

DEVELOPMENT = True

raw_filepath = "data/raw/"
interim_filepath = "data/interim/"
external_filepath = "data/external/"
processed_filepath = "data/processed/"

/Users/vivianwang/Desktop/building_damage_estimates


In [2]:
raw_filepath = "data/raw/"
make_dataset.update_data(raw_filepath)

In [3]:
# Read in the data
war_fire_csv_path = "./data/raw/war_fires_by_ADM3.csv"
war_fire_df = pd.read_csv(war_fire_csv_path)

building_data_csv_path = "./data/raw/building_shapes_file.csv"
output_df = pd.read_csv(building_data_csv_path)

/var/folders/93/b531zk911js9h3xpnm3db4x00000gn/T/ipykernel_11215/2763408197.py:6: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  output_df = pd.read_csv(building_data_csv_path)


# Ukraine war fire dataset

In [4]:
war_fire_df.head()

,LATITUDE,LONGITUDE,ADM3_PCODE,municipality,ADM3_EN,ADM3_UA,ADM3_RU,ADM2_EN,ADM2_UA,ADM2_RU,ADM2_PCODE,ADM1_EN,ADM1_UA,ADM1_RU,ADM1_PCODE,ADM0_EN,year,ACQ_TIME,date,pop_exact
0,44.73999,34.29688,UA0120009,UA0120009,Izobilnenska,Ізобільненська,Изобильненский,Yaltynskyi,Ялтинський,Ялтинский,UA0120,Autonomous Republic of Crimea,Автономна Республіка Крим,Автономная Республика Крым,UA01,Ukraine,2023,3,2023-10-02,12.081634
1,44.85680,34.29169,UA0116005,UA0116005,Dobrivska,Добрівська,Добровский,Simferopolskyi,Сімферопольський,Симферопольский,UA0116,Autonomous Republic of Crimea,Автономна Республіка Крим,Автономная Республика Крым,UA01,Ukraine,2023,1104,2023-10-14,289.232330
2,44.85862,34.30407,UA0116005,UA0116005,Dobrivska,Добрівська,Добровский,Simferopolskyi,Сімферопольський,Симферопольский,UA0116,Autonomous Republic of Crimea,Автономна Республіка Крим,Автономная Республика Крым,UA01,Ukraine,2023,1104,2023-10-14,60.394661
3,44.85981,34.29746,UA0116005,UA0116005,Dobrivska,Добрівська,Добровский,Simferopolskyi,Сімферопольський,Симферопольский,UA0116,Autonomous Republic of Crimea,Автономна Республіка Крим,Автономная Республика Крым,UA01,Ukraine,2023,1104,2023-10-14,103.530571
4,44.86233,34.30030,UA0116005,UA0116005,Dobrivska,Добрівська,Добровский,Simferopolskyi,Сімферопольський,Симферопольский,UA0116,Autonomous Republic of Crimea,Автономна Республіка Крим,Автономная Республика Крым,UA01,Ukraine,2023,1017,2023-10-27,60.394661


In [5]:
war_fire_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68191 entries, 0 to 68190
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   LATITUDE      68191 non-null  float64
 1   LONGITUDE     68191 non-null  float64
 2   ADM3_PCODE    68032 non-null  object 
 3   municipality  68032 non-null  object 
 4   ADM3_EN       68032 non-null  object 
 5   ADM3_UA       68032 non-null  object 
 6   ADM3_RU       68032 non-null  object 
 7   ADM2_EN       68032 non-null  object 
 8   ADM2_UA       68032 non-null  object 
 9   ADM2_RU       68032 non-null  object 
 10  ADM2_PCODE    68032 non-null  object 
 11  ADM1_EN       68032 non-null  object 
 12  ADM1_UA       68032 non-null  object 
 13  ADM1_RU       68032 non-null  object 
 14  ADM1_PCODE    68032 non-null  object 
 15  ADM0_EN       68032 non-null  object 
 16  year          68191 non-null  int64  
 17  ACQ_TIME      68191 non-null  int64  
 18  date          68191 non-nu

# Building Dataset

In [6]:
output_df.head()

,osm_id,buildingma,addrstreet,addrcity,building,addrhousen,office,addrfull,name,buildingle,source,geometry
0,4338650.0,NaN,NaN,NaN,construction,NaN,NaN,NaN,Льодовий стадіон,5,NaN,"POLYGON ((30.4658704 50.3739959, 30.4672303 50..."
1,4528325.0,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((35.4149579 45.3275258, 35.4150137 45..."
2,4528616.0,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((35.4150689 45.3277749, 35.4151686 45..."
3,4528933.0,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((35.4122859 45.3309186, 35.4123139 45..."
4,4529198.0,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((33.7198442 44.4086498, 33.720139 44...."


In [7]:
output_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6018387 entries, 0 to 6018386
Data columns (total 12 columns):
 #   Column      Dtype  
---  ------      -----  
 0   osm_id      float64
 1   buildingma  float64
 2   addrstreet  object 
 3   addrcity    object 
 4   building    object 
 5   addrhousen  object 
 6   office      object 
 7   addrfull    object 
 8   name        object 
 9   buildingle  object 
 10  source      object 
 11  geometry    object 
dtypes: float64(2), object(10)
memory usage: 551.0+ MB


# Reverse locate

In [8]:
# If DEVELOPMENT is True, only run on first 100 rows
if DEVELOPMENT:
    war_fire_df = war_fire_df[:1000]

# Write the same code but add tqdm to see the progress of the loop
for idx, row in tqdm(war_fire_df.iterrows(), total=war_fire_df.shape[0]):
    war_fire_df.loc[war_fire_df.index[idx], 'state'] = make_dataset.find_states(row)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [9]:
war_fire_df["state"].value_counts()

state
Odesa Oblast                     468
Autonomous Republic of Crimea    304
Republic of Crimea               194
                                  18
Kherson Oblast                    16
Name: count, dtype: int64

In [10]:
# Saving the war fire model back to a file
war_fire_df.to_csv("./data/processed/ukraine_war_fires_with_state_lookup.csv", index=False)

In [11]:
# Read processed war fire data
war_fire_df = pd.read_csv("./data/processed/ukraine_war_fires_with_state_lookup.csv")
war_fire_df["state"].value_counts()

state
Odesa Oblast                     468
Autonomous Republic of Crimea    304
Republic of Crimea               194
Kherson Oblast                    16
Name: count, dtype: int64

In [12]:
# Limit war fires to states of interest
war_fire_df_state = war_fire_df[war_fire_df["state"].isin(["Odesa Oblast","Chernihiv Oblast", "Sumy Oblast", "Kharkiv Oblast","Luhansk Oblast","Donetsk Oblast","Zaporizhia Oblast","Kherson Oblast","Mykolaiv Oblast"])]
war_fire_df_state.head() 

,LATITUDE,LONGITUDE,ADM3_PCODE,municipality,ADM3_EN,ADM3_UA,ADM3_RU,ADM2_EN,ADM2_UA,ADM2_RU,...,ADM1_EN,ADM1_UA,ADM1_RU,ADM1_PCODE,ADM0_EN,year,ACQ_TIME,date,pop_exact,state
223,45.30062,28.89845,UA5108003,UA5108003,Izmailska,Ізмаїльська,Измаильская,Izmailskyi,Ізмаїльський,Измаильский,...,Odeska,Одеська,Одесская,UA51,Ukraine,2023,48,2023-08-02,280.977295,Odesa Oblast
227,45.30187,28.90038,UA5108009,UA5108009,Safianivska,Саф'янівська,Сафьяновская,Izmailskyi,Ізмаїльський,Измаильский,...,Odeska,Одеська,Одесская,UA51,Ukraine,2023,1155,2023-08-03,316.327911,Odesa Oblast
232,45.30349,28.89518,UA5108003,UA5108003,Izmailska,Ізмаїльська,Измаильская,Izmailskyi,Ізмаїльський,Измаильский,...,Odeska,Одеська,Одесская,UA51,Ukraine,2023,30,2023-08-03,280.977295,Odesa Oblast
234,45.30404,28.89320,UA5108003,UA5108003,Izmailska,Ізмаїльська,Измаильская,Izmailskyi,Ізмаїльський,Измаильский,...,Odeska,Одеська,Одесская,UA51,Ukraine,2023,120,2023-08-03,280.977295,Odesa Oblast
241,45.30515,28.90079,UA5108009,UA5108009,Safianivska,Саф'янівська,Сафьяновская,Izmailskyi,Ізмаїльський,Измаильский,...,Odeska,Одеська,Одесская,UA51,Ukraine,2023,48,2023-08-02,316.327911,Odesa Oblast


In [13]:
war_fire_df_state["state"].value_counts()

state
Odesa Oblast      468
Kherson Oblast     16
Name: count, dtype: int64

In [14]:
# Add in the geometry column
war_fire_df_state['geometry'] = war_fire_df_state.apply(lambda row: Point(row['LATITUDE'], row['LONGITUDE']), axis=1)
war_fire_df_state["geometry"].head()

/var/folders/93/b531zk911js9h3xpnm3db4x00000gn/T/ipykernel_11215/3676197505.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  war_fire_df_state['geometry'] = war_fire_df_state.apply(lambda row: Point(row['LATITUDE'], row['LONGITUDE']), axis=1)


223    POINT (45.30062 28.89845)
227    POINT (45.30187 28.90038)
232    POINT (45.30349 28.89518)
234     POINT (45.30404 28.8932)
241    POINT (45.30515 28.90079)
Name: geometry, dtype: object

# calculate UTM

In [ ]:

# change to GeoDataFrame
# geo_build_df = gpd.GeoDataFrame(buildings_df)
# geo_build_df = geo_build_df.set_geometry("geometry_wkt_cent")

In [ ]:
# from pyproj import Proj
# pp = Proj(proj='utm',zone=10,ellps='WGS84', preserve_units=False)

# geo_build_df["xx"], geo_build_df["yy"]= pp(geo_build_df["geometry_wkt_cent"].x.values,geo_build_df["geometry_wkt_cent"].y.values)
# # My_data["X"] = xx
# # My_data["Y"] = yy 

In [ ]:
# First, convert WKT column to geometries
#buildings_df_state['geometry_wkt_cent2'] = gpd.GeoSeries.from_wkt(buildings_df_state['geometry_wkt_cent'])

# Then create a GeoDataFrame, setting the geometry to the converted geometries
# geo_build_df = gpd.GeoDataFrame(buildings_df_state, geometry='geometry_wkt_cent')

In [ ]:
# geo_build_df.crs = 'epsg:32635'
# # change the projection of geodf
# geo_build_df_2 = geo_build_df.to_crs(epsg=32635)
# print(geo_build_df_2)

In [ ]:
#geo_build_df_2["geometry_wkt_cent"]

# Buffer 

In [ ]:
# import geopandas as gpd
# from shapely.geometry import Point
# from pprint import pprint

# buffer_radius_m = 500
# buffered_gdf = geo_build_df_2.copy()

# buffered_gdf['geometry_wkt_cent_buff'] = buffered_gdf.buffer(buffer_radius_m)  # 1 degree of latitude is approximately 111.32 km
# # Print the buffered GeoDataFrame

# pprint(buffered_gdf)

In [ ]:
#pip install pyproj --upgrade


In [15]:
import pyproj
print(pyproj.__version__)


3.6.1


### with utm conversion


In [16]:
# In summary, the code first ensures that the GeoDataFrame has a defined CRS, 
# estimates the UTM CRS, and then reprojects the data to that UTM CRS. 
# The code then applies a buffer of 50 meters around each geometry in the 
# UTM-projected GeoDataFrame. Finally the code sets the geometry of the GeoDataFrame to the buffered geometries.
geo_warfires_df = gpd.GeoDataFrame(war_fire_df_state, geometry='geometry')
geo_warfires_df.set_crs(epsg=4326, inplace=True)
utm_crs = geo_warfires_df.estimate_utm_crs()
geo_warfires_df_utm = geo_warfires_df.to_crs("EPSG:32635")

# Then, apply the buffer in meters
buffer_radius_m = 50  # Buffer radius in meters
geo_warfires_df_utm['geometry_buff'] = geo_warfires_df_utm.buffer(buffer_radius_m)
geo_warfires_df_utm = geo_warfires_df_utm.set_geometry("geometry_buff")
geo_warfires_df_utm['geometry_buff'].head()

223    POLYGON ((2300506.403 3338797.153, 2300506.162...
227    POLYGON ((2300596.466 3339036.818, 2300596.226...
232    POLYGON ((2300853.347 3338470.964, 2300853.106...
234    POLYGON ((2300944.466 3338254.432, 2300944.225...
241    POLYGON ((2300917.490 3339136.015, 2300917.250...
Name: geometry_buff, dtype: geometry

# Buildings dataset

## Convert string geometry to integer

In [17]:
output_df['geometry_wkt'] = output_df['geometry'].apply(lambda x: loads(x) if x is not None else None)

In [18]:
buildings_df = output_df.copy()
buildings_df.head()

,osm_id,buildingma,addrstreet,addrcity,building,addrhousen,office,addrfull,name,buildingle,source,geometry,geometry_wkt
0,4338650.0,NaN,NaN,NaN,construction,NaN,NaN,NaN,Льодовий стадіон,5,NaN,"POLYGON ((30.4658704 50.3739959, 30.4672303 50...","POLYGON ((30.4658704 50.3739959, 30.4672303 50..."
1,4528325.0,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((35.4149579 45.3275258, 35.4150137 45...","POLYGON ((35.4149579 45.3275258, 35.4150137 45..."
2,4528616.0,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((35.4150689 45.3277749, 35.4151686 45...","POLYGON ((35.4150689 45.3277749, 35.4151686 45..."
3,4528933.0,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((35.4122859 45.3309186, 35.4123139 45...","POLYGON ((35.4122859 45.3309186, 35.4123139 45..."
4,4529198.0,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((33.7198442 44.4086498, 33.720139 44....","POLYGON ((33.7198442 44.4086498, 33.720139 44...."


In [ ]:
print(buildings_df["buildingle"].isna().sum()/len(buildings_df))

0.845716634706276


In [19]:
buildings_df["geometry_wkt_cent"] = np.nan

In [20]:
# buildings_df["geometry_wkt_cent"] = buildings_df["geometry_wkt"].apply(lambda x: x.centroid )
buildings_df['geometry_wkt_cent'] = buildings_df['geometry_wkt'].apply(lambda geom: Point(geom.centroid.y, geom.centroid.x))

# This will result in a column 'centroid' with tuples where the first element is latitude and the second is longitude.

In [21]:
buildings_df["geometry_wkt_cent"]

0          POINT (50.374100145135635 30.467099672291766)
1           POINT (45.32753752984076 35.415019664900306)
2           POINT (45.327807102299445 35.41514065136904)
3                  POINT (45.3309117 35.412328200000005)
4            POINT (44.40867430282276 33.72001963688109)
                               ...                      
6018382       POINT (49.2044672808804 25.60430534762094)
6018383    POINT (49.204496717246315 25.604121257448696)
6018384     POINT (49.20462673448506 25.604146556830067)
6018385           POINT (49.19898575 25.603685750000004)
6018386      POINT (49.19837142702208 25.60360779391183)
Name: geometry_wkt_cent, Length: 6018387, dtype: object

## Find the intersection of war_fires and buildings


### Buildings dataset with utm conversion


In [22]:
# The main purpose of this code is to convert the geometries in the building GeoDataFrame 
# from the original geographic coordinate system (WGS 84) to a UTM projected coordinate 
#  system (EPSG 32635). This might be useful for spatial analysis or visualization in a 
# local coordinate system, which can be more suitable for certain tasks.
geo_buildings_df = gpd.GeoDataFrame(buildings_df, geometry='geometry_wkt_cent')
geo_buildings_df.set_crs(epsg=4326, inplace=True)
geo_buildings_df_utm = geo_buildings_df.to_crs("EPSG:32635")

### How many buildings are within 50 meters of warfires

In [23]:
# Function to count buildings within a given buffer polygon
def count_buildings_in_buffer(buffer, buildings_gdf):
    # Use the within method to check if buildings are within the buffer zone
    return buildings_gdf.within(buffer).sum()

# Rewrite the code above using tqdm to see the progress of the loop
for idx, row in tqdm(geo_warfires_df_utm.iterrows(), total=geo_warfires_df_utm.shape[0]):
    geo_warfires_df_utm.loc[idx, 'building_count'] = count_buildings_in_buffer(row['geometry_buff'], geo_buildings_df_utm)

  0%|          | 0/484 [00:00<?, ?it/s]

In [24]:
geo_warfires_df_utm['geometry_buff']

223    POLYGON ((2300506.403 3338797.153, 2300506.162...
227    POLYGON ((2300596.466 3339036.818, 2300596.226...
232    POLYGON ((2300853.347 3338470.964, 2300853.106...
234    POLYGON ((2300944.466 3338254.432, 2300944.225...
241    POLYGON ((2300917.490 3339136.015, 2300917.250...
                             ...                        
995    POLYGON ((2258724.465 3979785.900, 2258724.224...
996    POLYGON ((2258870.590 3979739.320, 2258870.349...
997    POLYGON ((2259106.124 3978606.275, 2259105.883...
998    POLYGON ((2367623.799 3381580.882, 2367623.558...
999    POLYGON ((2347862.418 3497517.736, 2347862.177...
Name: geometry_buff, Length: 484, dtype: geometry

In [25]:
# Take a look at buildings that have been affected by war fires
geo_warfires_df_utm[geo_warfires_df_utm['building_count']  > 0].head()

,LATITUDE,LONGITUDE,ADM3_PCODE,municipality,ADM3_EN,ADM3_UA,ADM3_RU,ADM2_EN,ADM2_UA,ADM2_RU,...,ADM1_PCODE,ADM0_EN,year,ACQ_TIME,date,pop_exact,state,geometry,geometry_buff,building_count
242,45.30531,28.89597,UA5108003,UA5108003,Izmailska,Ізмаїльська,Измаильская,Izmailskyi,Ізмаїльський,Измаильский,...,UA51,Ukraine,2023,30,2023-08-03,280.977295,Odesa Oblast,POINT (2300971.250 3338590.022),"POLYGON ((2301021.250 3338590.022, 2301021.009...",3.0
243,45.30536,28.89639,UA5108003,UA5108003,Izmailska,Ізмаїльська,Измаильская,Izmailskyi,Ізмаїльський,Измаильский,...,UA51,Ukraine,2023,120,2023-08-03,280.977295,Odesa Oblast,POINT (2300968.613 3338638.623),"POLYGON ((2301018.613 3338638.623, 2301018.372...",3.0
422,45.43291,29.27516,UA5108005,UA5108005,Kiliiska,Кілійська,Килийская,Izmailskyi,Ізмаїльський,Измаильский,...,UA51,Ukraine,2023,113,2023-09-07,254.374786,Odesa Oblast,POINT (2306757.697 3383800.898),"POLYGON ((2306807.697 3383800.898, 2306807.456...",1.0
464,45.46016,29.29778,UA5108005,UA5108005,Kiliiska,Кілійська,Килийская,Izmailskyi,Ізмаїльський,Измаильский,...,UA51,Ukraine,2023,1051,2023-10-12,178.179245,Odesa Oblast,POINT (2309055.847 3386818.556),"POLYGON ((2309105.847 3386818.556, 2309105.606...",1.0


In [26]:
# Count the total number of buildings in each state
geo_warfires_df_utm.groupby('state')['building_count'].sum()

state
Kherson Oblast    0.0
Odesa Oblast      8.0
Name: building_count, dtype: float64

In [27]:
geo_warfires_df_utm.to_csv("./data/processed/buildings_impacted_by_war_fires.csv", index=False)

In [28]:
# Convert to a python script and save in local folder
!jupyter nbconvert --to script ./notebooks/*.ipynb --output-dir='./notebooks/python_scripts/python_scripts'

[NbConvertApp] Converting notebook ./notebooks/24_01_22-hh-buildings_impacted_by_war_fires.ipynb to script
/Users/vivianwang/anaconda3/lib/python3.11/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)
[NbConvertApp] Writing 8624 bytes to notebooks/python_scripts/python_scripts/24_01_22-hh-buildings_impacted_by_war_fires.py
[NbConvertApp] Converting notebook ./notebooks/24_01_22-hh-visualising_building_count.ipynb to script
/Users/vivianwang/anaconda3/lib/python3.11/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your note